## TRAIN.CSV
* 7985 unique tour_ids and 1934 unique biker_ids
* 13866 asked (3370 likes and 473 dislikes) - 3843 reviews - 10023 unanswered
* 3843 reviews were given by 1934 bikers. All bikers liked atleast 1 tour. Only 99 out of 1934 bikers disliked atleast 1 tour. 
* 584 questions to invited bikers. This involves 290 bikers (they were invited for at least 1 tour). A biker invited for one tour may not be invited for another tour. Only 2 bikers were invited to all the tours they were asked questions on.
* BG52448351 invited for 6 tours
* CE44012189 invited for 6 tours
* Some bikers were asked questions of same tour multiple times.
* 27th April 2012 to 12th December 2012

## BIKERS.CSV
* Getting language, location, age as of now.

## TOURS.CSV
* 

## TEST.CSV
* 2690 test queries - 1996 unique tour_ids, 297 unique bikers
* 1358 unseen tours in test dataset. So better use features from tours.csv

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import geopy
from geopy.geocoders import Nominatim
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from geopy.distance import geodesic
import sklearn
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import lightgbm as lgb
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, roc_auc_score, average_precision_score, accuracy_score
from sklearn.model_selection import train_test_split, KFold
from catboost import CatBoostClassifier, Pool, CatBoostRegressor
#from bayes_opt import BayesianOptimization

import warnings
warnings.filterwarnings("ignore")

In [ ]:
samp_sub = pd.read_csv('/kaggle/input/prml-data-contest-nov-2020/sample_submission.csv')
samp_sub.head()

In [ ]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)
pd.set_option('display.max_colwidth', -1)

In [ ]:
train_df = pd.read_csv('/kaggle/input/prml-data-contest-nov-2020/train.csv')

In [ ]:
train_df['like'].count()

* age filled with average age.
* member_since filled with dec 31
* gender filled with male
* timezone most frequent timezone
* latitude and longitude filled of US most

In [ ]:
a = {'latitude': {'FR': 46.2276, 'US': 37.0902, 'ID': -0.7893, 'IR': 32.4279, 'GB': 55.3781, 'CN': 35.8617,
              'LA': 19.8563, 'AR': -38.4161, 'ES': 40.4637, 'RU': 61.524, 'MY': 4.2105, 'KH': 12.5657,
              'GE': 42.3154, 'IT': 41.8719, 'RO': 45.9432, 'PH': 12.8797, 'CA': 56.1304, 'KR': 35.9078,
              'BR': -14.235, 'HK': 22.3964, 'HU': 47.1625, 'TR': 38.9637, 'TW':23.6978},
 'longitude': {'FR': 2.2137, 'US': -95.7129, 'ID': 113.9213, 'IR': 53.688, 'GB': -3.436, 'CN': 104.1954,
               'LA': 102.4955, 'AR': -63.6167, 'ES': -3.7492, 'RU': 105.3188, 'MY': 101.9758, 'KH': 104.991,
               'GE': 43.3569, 'IT': 12.5674, 'RO': 24.9668, 'PH': 121.774, 'CA': -106.3468, 'KR': 127.7669,
               'BR': -51.9253, 'HK': 114.1095, 'HU': 19.5033, 'TR': 35.2433, 'TW':120.9605}}
country_df = pd.DataFrame(a)
country_df

In [ ]:
train_df = pd.read_csv('/kaggle/input/prml-data-contest-nov-2020/train.csv')
test_df = pd.read_csv('/kaggle/input/prml-data-contest-nov-2020/test.csv')
bikers_df = pd.read_csv('/kaggle/input/prml-data-contest-nov-2020/bikers.csv')
tours_df = pd.read_csv('/kaggle/input/prml-data-contest-nov-2020/tours.csv')
bikers_network_df = pd.read_csv('/kaggle/input/prml-data-contest-nov-2020/bikers_network.csv')
tour_convoy_df = pd.read_csv('/kaggle/input/prml-data-contest-nov-2020/tour_convoy.csv')

def conv_to_int(x):
    try:
        val = 2012-int(x)
    except:
        val = -1
    return val
    
# preprocessing train and test
train_df.drop_duplicates(subset=['biker_id','tour_id'],inplace=True, ignore_index=True)
test_df.drop_duplicates(subset=['biker_id','tour_id'],inplace=True, ignore_index=True)

# preprocessing bikers_df
bikers_df['member_since'].fillna('31-12-2012', inplace=True)
bikers_df['member_since'].replace('--None', '31-12-2012', inplace=True)
bikers_df['member_since'] = pd.to_datetime(bikers_df['member_since'], format='%d-%m-%Y')
bikers_df['age'] = bikers_df['bornIn'].apply(lambda x:conv_to_int(x))
bikers_df.loc[bikers_df['age']==-1,'age'] = round(bikers_df[bikers_df['age']!=-1]['age'].mean())
bikers_df.set_index('biker_id', drop=True, inplace=True)
bikers_df['gender'] = bikers_df['gender'].apply(lambda x:0 if x=='female' else 1)
bikers_df['time_zone'].fillna(420, inplace=True)
bikers_df.drop(labels=['bornIn','area'], axis=1, inplace=True)
print('Preprocessed bikers.csv')

# preprocessing tours_df
tours_df.set_index('tour_id', drop=True, inplace=True)
grouped_df = tours_df.groupby('biker_id')[['latitude','longitude']]
lati_group_df = grouped_df.apply(lambda x:list(x['latitude'].unique())[0])
longi_group_df = grouped_df.apply(lambda x:list(x['longitude'].unique())[0])
print('Preprocessed tours.csv')

# preprocessing bikers_network_df and tour_convoy_df
bikers_network_df.set_index('biker_id', drop=True, inplace=True)
bikers_network_df.fillna('', inplace=True)
print('Preprocessed bikers_network.csv')
tour_convoy_df.set_index('tour_id', drop=True, inplace=True)
tour_convoy_df.fillna('', inplace=True)
print('Preprocessed tour_convoy.csv')

In [ ]:
def latitude_func_(country_id):
    if country_id=='PI':
        country_id='PH'
    return country_df.loc[country_id,'latitude']

def longitude_func_(country_id):
    if country_id=='PI':
        country_id='PH'
    return country_df.loc[country_id,'longitude']

def full_preprocess(df_to, save_name):
    # features from bikers_df
    temp_df = df_to.set_index('biker_id', drop=True).join(bikers_df).reset_index()
    temp_df['timestamp'] = pd.to_datetime(temp_df['timestamp'], format='%d-%m-%Y  %H:%M:%S')
    temp_df['join_query_difference'] = (temp_df['timestamp']-temp_df['member_since']).dt.days
    temp_df['month_joined'] = temp_df['member_since'].dt.month
    temp_df['day_joined'] = temp_df['member_since'].dt.day
    temp_df['month_query'] = temp_df['timestamp'].dt.month
    temp_df['day_query'] = temp_df['timestamp'].dt.day
    temp_df['hour_query'] = temp_df['timestamp'].dt.hour
    temp_df['minute_query'] = temp_df['timestamp'].dt.minute
    temp_df['biker_latitude'] = temp_df['location_id'].apply(latitude_func_)
    temp_df['biker_longitude'] = temp_df['location_id'].apply(longitude_func_)

    # features from tours_df
    temp_df = temp_df.set_index('tour_id', drop=True).join(tours_df.loc[:,'biker_id'],rsuffix='_organizer').reset_index()
    temp_df = temp_df.set_index('tour_id', drop=True).join(tours_df.loc[:,'city':'longitude'],rsuffix='_tour_').reset_index()
    temp_df = temp_df.set_index('tour_id', drop=True).join(tours_df.loc[:,'w1':'w_other']).reset_index()
    temp_df = temp_df.set_index('tour_id', drop=True).join(tours_df.loc[:,'tour_date']).reset_index()
    temp_df.loc[temp_df['latitude'].isna(),'latitude'] = temp_df.loc[temp_df['latitude'].isna(),
                                                                     'biker_id_organizer'].apply(lambda x:lati_group_df[x])
    temp_df.loc[temp_df['longitude'].isna(),'longitude'] = temp_df.loc[temp_df['longitude'].isna(),
                                                                     'biker_id_organizer'].apply(lambda x:longi_group_df[x])
    set1 = set(temp_df.loc[temp_df['latitude'].isna(),'biker_id_organizer'].unique())
    set2 = set(bikers_df.index.unique())
    set3 = set(tours_df.loc[(tours_df['latitude'].isna())&(~tours_df['country'].isna()),:].index.unique())
    set4 = set(temp_df.loc[temp_df['latitude'].isna(),'tour_id'].unique())
    print(bikers_df.loc[set1.intersection(set2),'location_id'])
    for b_id in set1.intersection(set2):
        country_id = bikers_df.loc[b_id, 'location_id']
        lati = latitude_func_(country_id)
        longi = longitude_func_(country_id)
        temp_df.loc[(temp_df['latitude'].isna())&(temp_df['biker_id_organizer']==b_id),['latitude','longitude']] = [lati, longi]
    temp_df['latitude'] = temp_df['latitude'].fillna(latitude_func_('US'))
    temp_df['longitude'] = temp_df['longitude'].fillna(longitude_func_('US'))
    temp_df.drop(labels=['city','state','pincode','country'],axis=1, inplace=True)
    temp_df['tour_date'] = pd.to_datetime(temp_df['tour_date'], format='%d-%m-%Y')
    temp_df['join_tour_difference'] = (temp_df['tour_date']-temp_df['member_since']).dt.days
    temp_df['year_tour'] = temp_df['tour_date'].dt.year
    temp_df['month_tour'] = temp_df['tour_date'].dt.month
    temp_df['day_tour'] = temp_df['tour_date'].dt.day
    temp_df['tour_query_difference'] = (temp_df['timestamp']-temp_df['tour_date']).dt.days
#     temp_df['biker_code_fl'] = temp_df['biker_id'].apply(lambda x:str(x)[0])
#     temp_df['biker_code_sl'] = temp_df['biker_id'].apply(lambda x:str(x)[1])
#     temp_df['biker_code'] = temp_df['biker_id'].apply(lambda x:str(x)[:2])
#     temp_df['tour_code_fl'] = temp_df['tour_id'].apply(lambda x:str(x)[1])
#     temp_df['tour_code_sl'] = temp_df['tour_id'].apply(lambda x:str(x)[1])
#     temp_df['tour_code'] = temp_df['tour_id'].apply(lambda x:str(x)[:2])

    # getting the order of tours for a given biker
    num_queries = temp_df.groupby('biker_id').count()['tour_id'].to_frame()
    num_queries.rename(columns={'tour_id':'num_queries_count'}, inplace=True)
    temp_df = temp_df.join(num_queries, on='biker_id')
    grouped = temp_df.groupby('biker_id')
    concat_list = []
    for i, df_sub in grouped:
        df_sub = df_sub.sort_values(by=['timestamp','tour_date'], ignore_index=True)
        df_sub['timestamp_index'] = df_sub.index/df_sub.index.max()
        df_sub = df_sub.sort_values(by=['tour_date','timestamp'], ignore_index=True)
        df_sub['tour_date_index'] = df_sub.index/df_sub.index.max()
        concat_list.append(df_sub)
    temp_df = pd.concat(concat_list, ignore_index=True)

    # getting order of biker for a given tour
    combined_df = pd.concat([train_df[['biker_id','tour_id','timestamp','like','dislike']],
                             test_df[['biker_id','tour_id','timestamp']]],join='outer').reset_index(drop=True)
    combined_df['like'] = combined_df['like'].fillna(0)
    combined_df['dislike'] = combined_df['dislike'].fillna(0)
    combined_df['timestamp'] = pd.to_datetime(combined_df['timestamp'], format='%d-%m-%Y  %H:%M:%S')
    combined_df = combined_df.join(bikers_df.loc[:,'member_since'], on='biker_id').reset_index(drop=True)
    num_bikers_asked_total = combined_df.groupby('tour_id').count()['biker_id'].to_frame()
    num_bikers_asked_total.rename(columns={'biker_id':'num_bikers_asked_total'}, inplace=True)
    num_bikers_asked_train = train_df.groupby('tour_id').count()['biker_id'].to_frame()
    num_bikers_asked_train.rename(columns={'biker_id':'num_bikers_asked_train'}, inplace=True)
    num_bikers_like_dislike = combined_df.groupby('tour_id').sum()[['like','dislike']]
    num_bikers_like_dislike.rename(columns={'like':'num_bikers_liked', 'dislike':'num_bikers_disliked'}, 
                                   inplace=True)

    combined_df = combined_df.join(num_bikers_asked_total, on='tour_id')
    combined_df = combined_df.join(num_bikers_asked_train, on='tour_id')
    combined_df = combined_df.join(num_bikers_like_dislike, on='tour_id')
    combined_df.fillna(0, inplace=True)
    combined_df['percent_train_like'] = combined_df['num_bikers_liked']/combined_df['num_bikers_asked_train']
    combined_df['percent_train_dislike'] = combined_df['num_bikers_disliked']/combined_df['num_bikers_asked_train']
    combined_df.fillna(0, inplace=True)
    grouped2 = combined_df.groupby('tour_id')
    concat_list2 = []
    for i, df_sub in grouped2:
        df_sub = df_sub.sort_values(by=['timestamp','member_since'], ignore_index=True)
        df_sub['biker_timestamp_index'] = (df_sub.index+1)/(df_sub.index.max()+1)
        df_sub = df_sub.sort_values(by=['member_since','timestamp'], ignore_index=True)
        df_sub['biker_member_since_index'] = (df_sub.index+1)/(df_sub.index.max()+1)
        concat_list2.append(df_sub)
    combined_df = pd.concat(concat_list2, ignore_index=True)
    combined_df.drop(columns=['timestamp','like','dislike','member_since'], inplace=True)
    assert(combined_df.isna().sum().sum()==0)
    temp_df = pd.merge(temp_df, combined_df, how='left', on=['biker_id','tour_id'])

    # from bikers_network_df and tour_convoy_df
    temp_df = temp_df.set_index('biker_id', drop=True).join(bikers_network_df.loc[:,'friends']).reset_index()
    temp_df = temp_df.set_index('tour_id', drop=True).join(tour_convoy_df.loc[:,:],rsuffix='_tour').reset_index()

    temp_df['friends'] = temp_df['friends'].apply(lambda x:set(str(x).split()))
    temp_df['going'] = temp_df['going'].apply(lambda x:set(str(x).split()))
    temp_df['maybe'] = temp_df['maybe'].apply(lambda x:set(str(x).split()))
    temp_df['invited_tour'] = temp_df['invited_tour'].apply(lambda x:set(str(x).split()))
    temp_df['not_going'] = temp_df['not_going'].apply(lambda x:set(str(x).split()))

    biker_loc_dict = bikers_df.loc[:,'location_id'].to_dict()

    def mapper_biker_to_loc(x0, x1):
        count = 0
        for i in x0:
            try:
                if biker_loc_dict[i]==x1:
                    count+=1
            except:
                pass
        return count

    temp_df['friends_loc'] = temp_df[['location_id','friends']].apply(lambda x:mapper_biker_to_loc(x[1],x[0]), axis=1)
    temp_df['going_loc'] = temp_df[['location_id','going']].apply(lambda x:mapper_biker_to_loc(x[1],x[0]), axis=1)
    temp_df['maybe_loc'] = temp_df[['location_id','maybe']].apply(lambda x:mapper_biker_to_loc(x[1],x[0]), axis=1)
    temp_df['invited_tour_loc'] = temp_df[['location_id','invited_tour']].apply(lambda x:mapper_biker_to_loc(x[1],x[0]), axis=1)
    temp_df['not_going_loc'] = temp_df[['location_id','not_going']].apply(lambda x:mapper_biker_to_loc(x[1],x[0]), axis=1)

    temp_df['num_friends'] = temp_df['friends'].apply(lambda x:len(x))
    temp_df['num_going'] = temp_df['going'].apply(lambda x:len(x))
    temp_df['num_maybe'] = temp_df['maybe'].apply(lambda x:len(x))
    temp_df['num_invited'] = temp_df['invited_tour'].apply(lambda x:len(x))
    temp_df['num_not_going'] = temp_df['not_going'].apply(lambda x:len(x))

    temp_df['friends_going'] = temp_df[['friends','going']].apply(lambda x:len(x[0].intersection(x[1])), axis=1)
    temp_df['friends_maybe'] = temp_df[['friends','maybe']].apply(lambda x:len(x[0].intersection(x[1])), axis=1)
    temp_df['friends_invited'] = temp_df[['friends','invited_tour']].apply(lambda x:len(x[0].intersection(x[1])), axis=1)
    temp_df['friends_not_going'] = temp_df[['friends','not_going']].apply(lambda x:len(x[0].intersection(x[1])), axis=1)
    temp_df['friend_organizer'] = temp_df[['friends','biker_id_organizer']].apply(lambda x:1 if str(x[0]) in x[1] else 0, axis=1)

    temp_df['percent_going'] = temp_df['friends_going']/temp_df['num_friends']
    temp_df['percent_maybe'] = temp_df['friends_maybe']/temp_df['num_friends']
    temp_df['percent_invited'] = temp_df['friends_invited']/temp_df['num_friends']
    temp_df['percent_not_going'] = temp_df['friends_not_going']/temp_df['num_friends']

    temp_df['percent_going_2'] = temp_df['friends_going']/temp_df['num_going']
    temp_df['percent_maybe_2'] = temp_df['friends_maybe']/temp_df['num_maybe']
    temp_df['percent_invited_2'] = temp_df['friends_invited']/temp_df['num_invited']
    temp_df['percent_not_going_2'] = temp_df['friends_not_going']/temp_df['num_not_going']

    temp_df['percent_going_3'] = temp_df['going_loc']/temp_df['num_going']
    temp_df['percent_maybe_3'] = temp_df['maybe_loc']/temp_df['num_maybe']
    temp_df['percent_invited_3'] = temp_df['invited_tour_loc']/temp_df['num_invited']
    temp_df['percent_not_going_3'] = temp_df['not_going_loc']/temp_df['num_not_going']
    temp_df['percent_friends_3'] = temp_df['friends_loc']/temp_df['num_friends']

    temp_df.fillna(0, inplace=True)

    temp_df['is_organizer'] = (temp_df['biker_id']==temp_df['biker_id_organizer']).astype(int)
    temp_df['geodesic_distance'] = temp_df.loc[:,['latitude',
                                                  'longitude',
                                                  'biker_latitude',
                                                  'biker_longitude']].apply(lambda x:geodesic((x[0],
                                                                                               x[1]),
                                                                                              (x[2],
                                                                                               x[3])).miles, axis=1)


    temp_df = temp_df.drop(labels=['biker_id_organizer','friends','going','maybe','invited_tour','not_going'], axis=1)
    temp_df.drop(labels=['timestamp','member_since','tour_date'], axis=1, inplace=True)
    temp_df.to_csv('{}_data_cleaned_v10.csv'.format(save_name), index=False)
    assert(temp_df.isna().sum().sum()==0)
    assert(len(list(temp_df.columns))==len(set(temp_df.columns)))
    return temp_df
train_cleaned_df = full_preprocess(train_df, 'train')
train_cleaned_df.sample(frac=1).head()

In [ ]:
test_cleaned_df = full_preprocess(test_df, 'test')
test_cleaned_df.sample(frac=1).head()

In [ ]:
!rm -r *

In [ ]:
EXP = 59
NUM_FOLDS = 5
CLASSIFIER_NAME = ['LGB']
ENC1 = 'ordinal'
USE_SCALER = True
SEED = 101
EXPO = 1
NUM_SEEDS = 2
NUM_LEAVES = 31
NUM_BOOST_ROUNDS = 20000
TRAIN_FULL = False
VERBOSE = False
DO_BALANCE = True
POSTPROCESS = True
POST_PROCESS2 = False

MULTICLASS = False
USE_TOUR_ID = False
BAYES_TUNING = False
INIT_ROUNDS = 5
OPT_ROUNDS = 15
NUM_FEATURES_BASE = 160 + int(USE_TOUR_ID)

if ENC1=='ordinal':
    NUM_FEATURES = NUM_FEATURES_BASE
if ENC1=='one_hot':
    NUM_FEATURES = NUM_FEATURES_BASE + 38

In [ ]:
train_cleaned_df = pd.read_csv('../input/train-data-cleaned/train_data_cleaned_v10.csv')
test_cleaned_df = pd.read_csv('../input/train-data-cleaned/test_data_cleaned_v10.csv')
drop_cols = ['percent_train_like','percent_train_dislike',
             'num_bikers_liked','num_bikers_disliked',
             'biker_code','biker_code_fl','biker_code_sl',
             'tour_code','tour_code_fl','tour_code_sl'
            ]
drop_cols += []
train_cleaned_df.drop(columns=drop_cols, inplace=True)
test_cleaned_df.drop(columns=drop_cols, inplace=True)

features = list(train_cleaned_df.copy().drop(labels=['like','dislike'], axis=1).columns)[2:]
cate_features = ['language_id','location_id']
if USE_TOUR_ID:
    cate_features = ['tour_id']+cate_features
    features = ['tour_id']+features
if MULTICLASS:
    train_cleaned_df['like'] = train_cleaned_df['like'] - train_cleaned_df['dislike'] +1
if ENC1=='ordinal':
    enc1 = OrdinalEncoder()
    X_temp = train_cleaned_df.loc[:,cate_features].values
    X_temp_test = test_cleaned_df.loc[:,cate_features].values
    X_temp_combined = np.concatenate([X_temp, X_temp_test], axis=0)
    enc1.fit(X_temp_combined)
    X_temp_enc = enc1.transform(X_temp)
    assert(X_temp_enc.shape[0]==(train_cleaned_df.shape[0]))
    train_cleaned_df.loc[:,cate_features] = X_temp_enc
elif ENC1=='one_hot':
    u1 = train_cleaned_df['language_id'].nunique()
    u2 = train_cleaned_df['location_id'].nunique()
    enc1 = OneHotEncoder(drop='if_binary')
    X_temp = train_cleaned_df.loc[:,['language_id','location_id']].values
    enc1.fit(X_temp)
    X_temp_enc = enc1.transform(X_temp).toarray()
    encoded_features1 = list(enc1.get_feature_names(['language_id','location_id']))
    assert(X_temp_enc.shape==(train_cleaned_df.shape[0],u1+u2))
    train_cleaned_df.loc[:,encoded_features1] = X_temp_enc
    train_cleaned_df.drop(labels=['language_id','location_id'], axis=1, inplace=True)
cate_features_ind = [features.index(i) for i in cate_features]
    
def get_data_preprocessed(train_cleaned_df, seed):
    cross_val_df = train_cleaned_df.copy().groupby('biker_id')['like'].sum().to_frame()
    cross_val_df.reset_index(inplace=True)
    cross_val_df['fold'] = -1
    kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=seed)
    for i, (_, val_index) in enumerate(kf.split(np.arange(cross_val_df.shape[0]))):
        cross_val_df.loc[val_index,'fold'] = i
    cross_val_df.set_index('biker_id', drop=True, inplace=True)
    train_cleaned_df = train_cleaned_df.join(cross_val_df.loc[:,'fold'], on='biker_id')
    train_cleaned_df['prediction_scores'] = -1
    if DO_BALANCE:
        subset = train_cleaned_df[train_cleaned_df['like']==1]
        train_cleaned_df = pd.concat([train_cleaned_df, subset, subset],
                                      ignore_index=True).sort_values(by=['tour_id','biker_id'], ignore_index=True)
    return train_cleaned_df

In [ ]:
def map_calculate(temp_df, dislike):
    """
    4 columns:biker_id, tour_id, like, prediction_scores
    drop duplicates first - drop negatives in true_label
    """
    temp_df = temp_df.drop_duplicates(ignore_index=True)
    grouped = temp_df.groupby('biker_id')
    map_score = 0; count = 0;
    for i,df in grouped:
        if dislike:
            true = df.loc[:,'dislike'].values
        else:
            true = df.loc[:,'like'].values
        if MULTICLASS:
            true -= 1
        true[true==-1] = 0
        pred = df.loc[:,'prediction_scores'].values
        if dislike:
            ordered_label = df.sort_values(by='prediction_scores', ascending=False)['dislike'].values
        else:
            ordered_label = df.sort_values(by='prediction_scores', ascending=False)['like'].values
        if true.sum()==0:
            continue
        ap_score = average_precision_score(true, pred)
        map_score += ap_score
        count+=1
    return map_score/count

def do_cv_lgb(train_cleaned_df, param, clf_name):
    clf_list = []
    enc2_list = []
    train_cleaned_df_train = train_cleaned_df.copy()
    train_cleaned_df_train['prediction_scores_sum'] = 0
    print(clf_name)
    #assert(len(features)==NUM_FEATURES)

    for i in range(NUM_FOLDS):   
        #print('Fold-->{}'.format(i))
        tour_sum = train_cleaned_df.loc[train_cleaned_df['fold']!=i,:].groupby('tour_id')['like','dislike'].sum()
        tour_sum.reset_index(inplace=True)
        tour_sum.columns = ['tour_id','like_sum','dislike_sum']
        x_df_temp = train_cleaned_df.copy().drop(labels=['like','dislike'], axis=1)
        X_train_df = x_df_temp.loc[x_df_temp['fold']!=i,:].drop(labels=['fold','prediction_scores'], axis=1)
        Y_like_train = train_cleaned_df.loc[train_cleaned_df['fold']!=i,'like'].values
        Y_dislike_train = train_cleaned_df.loc[train_cleaned_df['fold']!=i,'dislike'].values    
        if TRAIN_FULL:
            X_train_df = x_df_temp.loc[:,:].drop(labels=['fold','prediction_scores'], axis=1)
            Y_like_train = train_cleaned_df.loc[:,'like'].values
            Y_dislike_train = train_cleaned_df.loc[:,'dislike'].values    
        Y_like_val = train_cleaned_df.loc[train_cleaned_df['fold']==i,'like'].values
        Y_dislike_val = train_cleaned_df.loc[train_cleaned_df['fold']==i,'dislike'].values
        X_val_df = x_df_temp.loc[x_df_temp['fold']==i,:].drop(labels=['fold','prediction_scores'], axis=1)
        X_val_df.fillna(0, inplace=True)
        if USE_TOUR_ID:
            X_train = X_train_df.drop(labels=['biker_id'], axis=1).values
            X_val = X_val_df.drop(labels=['biker_id'], axis=1).values
        else:
            X_train = X_train_df.drop(labels=['tour_id','biker_id'], axis=1).values
            X_val = X_val_df.drop(labels=['tour_id','biker_id'], axis=1).values
        if USE_SCALER:
            enc2 = StandardScaler()
            enc2.fit(X_train)
            X_train_enc = enc2.transform(X_train)
            X_val_enc = enc2.transform(X_val)
            enc2_list.append(enc2)
        else:
            X_train_enc = X_train
            X_val_enc = X_val
        
        if clf_name=='LGB':
            train_data = lgb.Dataset(X_train_enc, label=Y_like_train)
            val_data = lgb.Dataset(X_val_enc, label=Y_like_val)
            bst = lgb.train(param, train_data, NUM_BOOST_ROUNDS, valid_sets=[val_data],
                            feature_name=features, categorical_feature=cate_features,
                            verbose_eval=VERBOSE, early_stopping_rounds=250)
            Y_pred = bst.predict(X_val_enc, num_iteration=bst.best_iteration)
            Y_pred_train = bst.predict(X_train_enc, num_iteration=bst.best_iteration)
            pred_scores = Y_pred
            pred_scores_train = Y_pred_train
        elif clf_name=='CATBOOST':
            train_data = Pool(data=X_train_enc, label=Y_like_train,cat_features=cate_features_ind)
            val_data = Pool(data=X_val_enc, label=Y_like_val,cat_features=cate_features_ind)
            bst = CatBoostClassifier(iterations=NUM_BOOST_ROUNDS,random_seed=SEED,
                                     eval_metric='AUC')
            bst.fit(train_data, eval_set=[val_data], use_best_model=True,
                    verbose_eval=VERBOSE, early_stopping_rounds=250)
            Y_pred = bst.predict_proba(X_val_enc)[:,1]
            Y_pred_train = bst.predict_proba(X_train_enc)[:,1]
            #Y_pred = bst.predict(X_val_enc)
            #Y_pred_train = bst.predict(X_train_enc)
            pred_scores = Y_pred
            pred_scores_train = Y_pred_train
        
        if MULTICLASS:
            pred_scores = Y_pred[:,2]-Y_pred[:,0]
            pred_scores_train = Y_pred_train[:,2]-Y_pred_train[:,0]
        train_cleaned_df.loc[train_cleaned_df['fold']==i,'prediction_scores'] = pred_scores
        if not TRAIN_FULL:
            train_cleaned_df_train.loc[train_cleaned_df['fold']!=i,'prediction_scores'] = pred_scores_train
            train_cleaned_df_train.loc[train_cleaned_df['fold']!=i,'prediction_scores_sum'] += pred_scores_train
        else:
            train_cleaned_df_train.loc[:,'prediction_scores'] = pred_scores_train
            train_cleaned_df_train.loc[:,'prediction_scores_sum'] += pred_scores_train
        dislike = False
        map_score = map_calculate(train_cleaned_df.loc[train_cleaned_df['fold']==i,
                                                       ['biker_id','tour_id','like', 'dislike',
                                                        'prediction_scores']], dislike)
        map_score_train = map_calculate(train_cleaned_df_train.loc[train_cleaned_df['fold']!=i,
                                                       ['biker_id','tour_id','like', 'dislike',
                                                        'prediction_scores']], dislike)
        print('Train-->{:.3f}, Val-->{:.3f}'.format(map_score_train, map_score))
        clf_list.append(bst)
        if TRAIN_FULL:
            break
    final_score = map_calculate(train_cleaned_df.loc[:,['biker_id','tour_id','like', 'prediction_scores']], dislike=False)
    train_cleaned_df_train['prediction_scores'] = train_cleaned_df_train['prediction_scores_sum']
    final_score_train = map_calculate(train_cleaned_df_train.loc[:,['biker_id','tour_id','like', 'prediction_scores']], dislike=False)
    return train_cleaned_df, clf_list, enc2_list, final_score, final_score_train

def bayes_parameter_opt_lgb(train_cleaned_df,init_round, opt_round, random_seed):
    
    def lgb_eval(learning_rate, num_leaves, feature_fraction, bagging_fraction, 
                 max_depth, max_bin, min_data_in_leaf,min_sum_hessian_in_leaf,subsample):
        param = {'metric':'auc'}
        param['learning_rate'] = max(min(learning_rate, 1), 0)
        param["num_leaves"] = int(round(num_leaves))
        param['feature_fraction'] = max(min(feature_fraction, 1), 0)
        param['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        param['max_depth'] = int(round(max_depth))
        param['max_bin'] = int(round(max_bin))
        param['min_data_in_leaf'] = int(round(min_data_in_leaf))
        param['min_sum_hessian_in_leaf'] = min_sum_hessian_in_leaf
        param['subsample'] = max(min(subsample, 1), 0)
        _, _, _, final_score = do_cv_lgb(train_cleaned_df, param)
        return final_score
    
    def lgb_eval_2(num_leaves):
        param = {'metric':'auc'}
        param["num_leaves"] = int(round(num_leaves))
        _, _, _, final_score, final_score_train = do_cv_lgb(train_cleaned_df, param)
        return final_score
     
    lgbBO = BayesianOptimization(lgb_eval_2, {'learning_rate': (0.001, 0.01),
                                            'num_leaves': (30.5, 100.5),
                                            #'feature_fraction': (0.1, 0.9),
                                            #'bagging_fraction': (0.8, 1),
                                            #'max_depth': (5, 30),
                                            #'max_bin':(20,90),
                                            #'min_data_in_leaf': (20, 80),
                                            #'min_sum_hessian_in_leaf':(0,100),
                                            #'subsample': (0.01, 1.0)
                                           }, 
                                 random_state=random_seed)

    lgbBO.maximize(init_points=init_round, n_iter=opt_round)
    model_auc=[]
    for model in range(len( lgbBO.res)):
        model_auc.append(lgbBO.res[model]['target'])
        
    return (lgbBO.res[pd.Series(model_auc).idxmax()]['target'], 
            lgbBO.res[pd.Series(model_auc).idxmax()]['params'])

def lgb_tune_and_train(train_cleaned_df, seed, clf_name):
    train_cleaned_df = get_data_preprocessed(train_cleaned_df, seed)
    if BAYES_TUNING:
        _, param = bayes_parameter_opt_lgb(train_cleaned_df,init_round=INIT_ROUNDS, 
                                           opt_round=OPT_ROUNDS, random_seed=seed)
        param['learning_rate'] = max(min(param['learning_rate'], 1), 0)
        param["num_leaves"] = int(round(param['num_leaves']))
        param['feature_fraction'] = max(min(param['feature_fraction'], 1), 0)
        param['bagging_fraction'] = max(min(param['bagging_fraction'], 1), 0)
        param['max_depth'] = int(round(param['max_depth']))
        param['max_bin'] = int(round(param['max_bin']))
        param['min_data_in_leaf'] = int(round(param['min_data_in_leaf']))
        param['min_sum_hessian_in_leaf'] = param['min_sum_hessian_in_leaf']
        param['subsample'] = max(min(param['subsample'], 1), 0)
        param['metric']='auc'
        #param['is_unbalance']=True
        #param['boost_from_average']=False
        train_cleaned_df, clf_list, enc2_list, final_score, final_score_train = do_cv_lgb(train_cleaned_df, param)
    else:
        param = {}
        param['metric'] = 'auc'
        param['num_leaves'] = NUM_LEAVES
        if MULTICLASS:
            param['objective'] = 'multiclass'
            param['metric']='multi_logloss'
            param['num_class']=3
        train_cleaned_df, clf_list, enc2_list, final_score, final_score_train = do_cv_lgb(train_cleaned_df, param, clf_name)
    print('Final Train-->{:.4f}, Val-->{:.4f}'.format(final_score_train, final_score))
    return train_cleaned_df, clf_list, enc2_list, final_score

clf_name_list = []
for s in range(NUM_SEEDS):
    for s2, clf_name in enumerate(CLASSIFIER_NAME):
        print(s)
        clf_name_list.append(clf_name)
        train_cleaned_df_sub, clf_sub, enc2_sub, final_score_sub = lgb_tune_and_train(train_cleaned_df,SEED+s, clf_name)
        if s==0 and s2==0:
            clf_list = clf_sub
            enc2_list = enc2_sub
            train_cleaned_df_final = train_cleaned_df_sub.copy().sort_values(by=['biker_id','tour_id'])
            train_cleaned_df_final['prediction_scores'] = train_cleaned_df_final['prediction_scores']**EXPO
        else:
            clf_list += clf_sub
            enc2_list += enc2_sub
            train_cleaned_df_final['prediction_scores'] += train_cleaned_df_sub.sort_values(by=['biker_id',
                                                                                                'tour_id'])['prediction_scores']**EXPO
        final_score = map_calculate(train_cleaned_df_final.loc[:,['biker_id','tour_id',
                                                            'like', 'prediction_scores']], dislike=False)
        print('{:.4f}'.format(final_score))
train_cleaned_df_final['prediction_scores'] /= NUM_SEEDS
train_cleaned_df_final[['biker_id','tour_id','like','dislike',
                  'prediction_scores']].to_csv('train_scores_exp{}-{:.3f}.csv'.format(EXP, final_score), index=False)

In [ ]:
print(len(clf_list))
try:
    print(len(clf_list2))
except:
    pass

### Predicting test

In [ ]:
test_cleaned_df = pd.read_csv('../input/train-data-cleaned/test_data_cleaned_v10.csv')
test_cleaned_df.drop(columns=drop_cols, inplace=True)
X_temp = test_cleaned_df.loc[:,cate_features].values

if ENC1=='ordinal':
    print('Ordinal encoder')
    X_temp_enc = enc1.transform(X_temp)
    test_cleaned_df.loc[:,cate_features] = X_temp_enc
elif ENC1=='one_hot':
    print('One hot encoder')
    X_temp_enc = enc1.transform(X_temp).toarray()
    encoded_features1 = list(enc1.get_feature_names(['language_id','location_id']))
    test_cleaned_df.loc[:,encoded_features1] = X_temp_enc
    test_cleaned_df.drop(labels=['language_id','location_id'], axis=1, inplace=True)
    
test_cleaned_df['prediction_scores'] = -1
if USE_TOUR_ID:
    X_test = test_cleaned_df.loc[:,:].drop(['biker_id','prediction_scores'],axis=1).values
else:
    X_test = test_cleaned_df.loc[:,:].drop(['biker_id','prediction_scores','tour_id'],axis=1).values

print(len(clf_list))
Y_pred = np.zeros(shape=(test_cleaned_df.shape[0],))
for cc in range(len(clf_list)):
    clf = clf_list[cc]
    if USE_SCALER:
        enc2 = enc2_list[cc]
        X_test_enc = enc2.transform(X_test)
    else:
        X_test_enc = X_test
    #assert(X_test_enc.shape==(test_cleaned_df.shape[0],NUM_FEATURES))
    if clf_name_list[cc//5]=='LGB':
        one_hot_predict = clf.predict(X_test_enc, num_iteration=clf.best_iteration)
        if MULTICLASS:
            Y_pred += one_hot_predict[:,2]-one_hot_predict[:,0]
        else:
            Y_pred += one_hot_predict
        continue
    elif clf_name_list[cc//5]=='CATBOOST':
        X_test_enc = Pool(X_test_enc, cat_features=cate_features_ind)
        one_hot_predict = clf.predict_proba(X_test_enc)[:,1]
        if MULTICLASS:
            Y_pred += one_hot_predict[:,2]-one_hot_predict[:,0]
        else:
            Y_pred += one_hot_predict
        continue
Y_pred_final = Y_pred/len(clf_list)
test_cleaned_df['prediction_scores'] = Y_pred_final
if ENC1=='ordinal':
    inversed = enc1.inverse_transform(test_cleaned_df.loc[:,cate_features].values)
    test_cleaned_df.loc[:,cate_features] = inversed
test_cleaned_df[['biker_id','tour_id',
         'prediction_scores']].to_csv('test_scores_exp{}-{:.3f}.csv'.format(EXP, final_score), index=False)
test_cleaned_df.head()

In [ ]:
def postprocessing_func(train_df, test_df):
    train_df = train_df.drop_duplicates(subset=['biker_id','tour_id'],ignore_index=True)
    test_df = test_df.drop_duplicates(subset=['biker_id','tour_id'],ignore_index=True)
    pp_df_list = []
    train_df = train_df.sort_values(by=['biker_id','tour_id'], ignore_index=True)
    Y_like = train_df.like.values
    fold_sorted = train_df.fold.values
    for jj in range(NUM_FOLDS):
        X_train = train_df.loc[train_df['fold']!=jj,:].reset_index(drop=True)
        X_val = train_df.loc[train_df['fold']==jj,:].drop(['like','dislike',
                                                           'fold'], axis=1).reset_index(drop=True)
        pp_df_list.append(get_similarity_df(X_train, X_val))
    pp_df = pd.concat(pp_df_list, ignore_index=True)
    pp_df = pp_df.sort_values(by=['biker_id','tour_id'])
    pp_df['like'] = Y_like
    pp_df['fold'] = fold_sorted
    pp_model_list = []
    for kk in range(NUM_FOLDS):
        X_train = pp_df.loc[pp_df['fold']!=kk,'prediction_scores_test':'same_tour']
        X_val = pp_df.loc[pp_df['fold']==kk,'prediction_scores_test':'same_tour']
        Y_like_train = pp_df.loc[pp_df['fold']!=kk,'like']
        Y_like_val = pp_df.loc[pp_df['fold']==kk,'like']
        train_data = lgb.Dataset(X_train, label=Y_like_train)
        val_data = lgb.Dataset(X_val, label=Y_like_val)
        param = {}
        param['metric'] = 'auc'
        param['objective'] = 'binary'
        param['num_leaves'] = 8
        bst = lgb.train(param, train_data, 10000, valid_sets=[val_data],
                        verbose_eval=False, early_stopping_rounds=250)
        pp_model_list.append(bst)
        Y_pred = bst.predict(X_val, num_iteration=bst.best_iteration)
        pp_df.loc[pp_df['fold']==kk,'prediction_scores'] = Y_pred
        score = map_calculate(pp_df.loc[pp_df['fold']==kk,:], dislike=False)
        print(score)
    print('Final:',map_calculate(pp_df, dislike=False))
    Y_pred = np.zeros(shape=(test_df.shape[0],))
    for bst in pp_model_list:
        pp_test_df = get_similarity_df(train_df,test_df)
        X_test = pp_test_df.loc[:,'prediction_scores_test':'same_tour']
        Y_pred += bst.predict(X_test, num_iteration=bst.best_iteration)
    pp_test_df['prediction_scores'] = Y_pred/len(pp_model_list)
    return pp_test_df

def get_similarity_df(train_df, test_df):
    try:
        X_temp = train_df.loc[:,cate_features].values
        X_temp_enc = enc1.transform(X_temp)
        train_df.loc[:,cate_features] = X_temp_enc
    except:
        pass
    try:
        X_temp = test_df.loc[:,cate_features].values
        X_temp_enc = enc1.transform(X_temp)
        test_df.loc[:,cate_features] = X_temp_enc
    except:
        pass
    X = train_df.drop(['biker_id','tour_id','like','dislike','fold','prediction_scores'], axis=1)
    Y = test_df.drop(['biker_id','tour_id','prediction_scores'], axis=1)
    enc2_pp = StandardScaler()
    enc2_pp.fit(X.values)
    X_enc = enc2_pp.transform(X.values)
    Y_enc = enc2_pp.transform(Y.values)
    res = cosine_distances(X_enc, Y_enc)
    res_arg_min = np.argmin(res, axis=0)
    res_sim_df = train_df.loc[res_arg_min,['biker_id','tour_id','like',
                                           'dislike','prediction_scores']+cate_features].reset_index(drop=True)
    res_val_min = np.amin(res, axis=0)
    res_diff_mean = np.mean(res, axis=0) - res_val_min
    res_diff_max = np.amax(res, axis=0)-res_val_min
    res_sorted = np.sort(res, axis=0)
    assert(np.all(res_sorted[0,:]==res_val_min))
    res_diff_sec = res_sorted[1,:]-res_sorted[0,:]
    pp_df = test_df.copy()[['biker_id','tour_id', 'prediction_scores']+cate_features]
    pp_df['res_diff_sec'] = res_diff_sec
    pp_df['res_val_min'] = res_val_min
    pp_df['res_diff_max'] = res_diff_max
    pp_df['res_diff_mean'] = res_diff_max
    pp_df = pp_df.join(res_sim_df, rsuffix='_train')
    pp_df['same_location'] = pp_df['location_id']==pp_df['location_id_train']
    pp_df['same_language'] = pp_df['language_id']==pp_df['language_id_train']
    
    #get num of common friends
    #pp_df = pp_df.merge(bikers_network_df, on='biker_id', how='left')
    #pp_df = pp_df.merge(bikers_network_df, left_on='biker_id_train', right_on='biker_id', how='left', suffixes=('','_train'))
    #pp_df['friends'] = pp_df['friends'].apply(lambda x:set(str(x).split()))
    #pp_df['friends_train'] = pp_df['friends_train'].apply(lambda x:set(str(x).split()))
    #pp_df['friends_common_num'] = pp_df[['friends_train','friends']].apply(lambda x:len(x[0].intersection(x[1])), axis=1)
    
    pp_df['same_tour'] = pp_df['tour_id']==pp_df['tour_id_train']
    pp_df = pp_df.drop(['tour_id_train','biker_id_train',
                        'language_id_train','location_id_train',
                        #'friends','friends_train'
                       ]+cate_features, axis=1)
    pp_df.rename(columns={'prediction_scores':'prediction_scores_test',
                          'like':'like_sim','dislike':'dislike_sim'}, inplace=True)
    return pp_df
if POSTPROCESS:
    bikers_network_df = pd.read_csv('/kaggle/input/prml-data-contest-nov-2020/bikers_network.csv')
    bikers_network_df.fillna('', inplace=True)
    pp_test_df = postprocessing_func(train_cleaned_df_final, test_cleaned_df)

In [ ]:
def postprocessing_func2(train_cleaned_df_final, test_cleaned_df):
    pp2_df = []
    for i in range(NUM_FOLDS):
        train_df = train_cleaned_df_final.drop_duplicates(subset=['biker_id','tour_id']).copy()
        test_df = test_cleaned_df.copy()
        train2_cols = features + ['biker_id', 'tour_id','prediction_scores']
        X_train_df = train_df.loc[train_df['fold']!=i,train2_cols+['like','dislike']]
        sub1 = X_train_df.groupby('tour_id').sum()[['like','dislike']]
        sub2 = X_train_df.groupby('tour_id').mean()[['like','dislike']]
        sub3 = X_train_df.groupby('tour_id').count()['biker_id'].to_frame()
        sub3.columns = ['count']
        sub = sub1.join(sub2,lsuffix='_sum',rsuffix='_mean')
        sub = sub.join(sub3)
        sub = sub[sub['count']>=4].reset_index()
        X_train_df = X_train_df.merge(sub, on='tour_id')
        X_train_df.dropna(inplace=True)
        X_val_df = train_df.loc[train_df['fold']==i,train2_cols+['like','dislike']]
        X_val_df = X_val_df.merge(sub, on='tour_id')
        X_val_df.dropna(inplace=True)
        X_val_df['new_preds'] = 0
        features_pp = features+['prediction_scores','like_sum','dislike_sum','like_mean','dislike_mean','count']
        train_data = Pool(data=X_train_df.loc[:,features_pp].values, 
                          label=X_train_df.loc[:,'like'].values, cat_features=cate_features_ind)
        val_data = Pool(data=X_val_df.loc[:,features_pp].values, 
                        label=X_val_df.loc[:,'like'].values, cat_features=cate_features_ind)
        bst = CatBoostClassifier(iterations=NUM_BOOST_ROUNDS,random_seed=SEED,eval_metric='AUC')
        bst.fit(train_data, eval_set=[val_data], use_best_model=True,
                verbose_eval=False, early_stopping_rounds=250)
        Y_pred = bst.predict_proba(X_val_df.loc[:,features_pp].values)[:,1]
        Y_pred_train = bst.predict_proba(X_train_df.loc[:,features_pp].values)[:,1]
        pred_scores = Y_pred
        pred_scores_train = Y_pred_train
        X_val_df['new_preds'] = pred_scores
        temp = X_val_df[['biker_id','tour_id','new_preds']].copy()
        train_df = train_df.merge(temp,how='left', on=['biker_id','tour_id'])
        train_df.fillna(0, inplace=True)
        train_df['prediction_scores'] = np.where(train_df['new_preds']==0, 
                                                 train_df['prediction_scores'], 
                                                 train_df['new_preds'])
        score = map_calculate(train_df.loc[train_df['fold']==i,
                                           ['biker_id','tour_id','like', 
                                            'prediction_scores']], dislike=False)
        print(score)
        pp2_df.append(train_df.loc[train_df['fold']==i,:])
    pp2_df = pd.concat(pp2_df)
    return pp2_df

if POST_PROCESS2:
    pp2_df = postprocessing_func2(train_cleaned_df_final, test_cleaned_df)
    pp2_df

In [ ]:
def get_sub_file(test_cleaned_df):
    bikers_set = test_cleaned_df['biker_id'].unique()
    bikers = []
    tours = []
    for biker in bikers_set:
        subset = test_cleaned_df.loc[test_cleaned_df['biker_id']==biker,['biker_id','tour_id','prediction_scores']]
        subset = subset.sort_values(by='prediction_scores', ascending=False).reset_index(drop=True)
        tour_list = list(subset['tour_id'])
        tour = " ".join(tour_list)
        bikers.append(biker)
        tours.append(tour)
    sample_submission =pd.DataFrame(columns=["biker_id","tour_id"])
    sample_submission["biker_id"] = bikers
    sample_submission["tour_id"] = tours
    sample_submission.to_csv("sub{}_{:.3f}_5fold.csv".format(EXP, final_score),index=False)
    print(sample_submission.shape)
    return sample_submission

if POSTPROCESS:
    print('Postprocessed')
    sample_submission = get_sub_file(pp_test_df)
else:
    sample_submission = get_sub_file(test_cleaned_df)
sample_submission.head()

In [ ]:
if 'LGB' in CLASSIFIER_NAME:
    importance = np.zeros(shape=(NUM_FEATURES,))
    for clf in clf_list:
        importance += clf.feature_importance()
    importance/=len(clf_list)
    features = list(test_cleaned_df.columns)[2:-1]
    features = ['tour_id'] + features
    importance_ordered = list(np.argsort(importance)[::-1])
    features_ordered = [(features[i], importance[i]) for i in importance_ordered]
    features_ordered

In [ ]:
if 'LGB' in CLASSIFIER_NAME:
    plt.plot(importance)
    plt.show()

In [ ]:
features_ordered

# 0.79097 - 0.744(val)

In [ ]:
EXP = 58
NUM_FOLDS = 5
CLASSIFIER_NAME = ['LGB']
ENC1 = 'ordinal'
USE_SCALER = False
SEED = 101
EXPO = 1
NUM_SEEDS = 25
NUM_LEAVES = 31
NUM_BOOST_ROUNDS = 20000
TRAIN_FULL = False
VERBOSE = False
DO_BALANCE = False
POSTPROCESS = False
POST_PROCESS2 = False

MULTICLASS = False
USE_TOUR_ID = False
BAYES_TUNING = False
INIT_ROUNDS = 5
OPT_ROUNDS = 15
NUM_FEATURES_BASE = 160 + int(USE_TOUR_ID)

if ENC1=='ordinal':
    NUM_FEATURES = NUM_FEATURES_BASE
if ENC1=='one_hot':
    NUM_FEATURES = NUM_FEATURES_BASE + 38

# 0.79824

In [ ]:
EXP = 59
NUM_FOLDS = 5
CLASSIFIER_NAME = ['LGB']
ENC1 = 'ordinal'
USE_SCALER = True
SEED = 101
EXPO = 1
NUM_SEEDS = 1
NUM_LEAVES = 31
NUM_BOOST_ROUNDS = 20000
TRAIN_FULL = False
VERBOSE = False
DO_BALANCE = True
POSTPROCESS = True
POST_PROCESS2 = False

MULTICLASS = False
USE_TOUR_ID = False
BAYES_TUNING = False
INIT_ROUNDS = 5
OPT_ROUNDS = 15
NUM_FEATURES_BASE = 160 + int(USE_TOUR_ID)

if ENC1=='ordinal':
    NUM_FEATURES = NUM_FEATURES_BASE
if ENC1=='one_hot':
    NUM_FEATURES = NUM_FEATURES_BASE + 38